In [7]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score,train_test_split, GridSearchCV, ParameterGrid, StratifiedKFold, RandomizedSearchCV, cross_val_predict
from sklearn.metrics import mean_squared_error,r2_score,roc_curve,auc,precision_recall_curve, accuracy_score,mean_absolute_error
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeRegressor,DecisionTreeClassifier
from sklearn.ensemble import VotingRegressor, VotingClassifier, StackingRegressor, StackingClassifier, GradientBoostingRegressor,GradientBoostingClassifier, BaggingRegressor,BaggingClassifier,RandomForestRegressor,RandomForestClassifier,AdaBoostRegressor,AdaBoostClassifier
from sklearn.linear_model import LinearRegression,LogisticRegression, LassoCV, RidgeCV, ElasticNetCV
from sklearn.neighbors import KNeighborsRegressor
import itertools as it
import time as time
import xgboost as xgb
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from pyearth import Earth
import warnings
warnings.filterwarnings('ignore')

In [2]:
X_train = pd.read_csv("X_train.csv", index_col=0)
X_test = pd.read_csv("X_test.csv", index_col=0)
y_train = pd.read_csv("y_train.csv", index_col=0)
y_test = pd.read_csv("y_test.csv", index_col=0)
cv = KFold(5, shuffle=True, random_state=12)

In [53]:
# MARS MODEL
ma = Earth(max_terms=50, max_degree=1)
ma_train_preds = cross_val_predict(ma, X_train, y_train, cv = cv)
ma_train_preds_df = pd.DataFrame(ma_train_preds)
ma_train_preds_df.to_csv("ma_train_preds.csv")

In [54]:
ma.fit(X_train, y_train)
ma_test_preds = ma.predict(X_test)
ma_test_preds_df = pd.DataFrame(ma_test_preds)
ma_test_preds_df.to_csv("ma_test_preds.csv")

In [52]:
# BAGGING MODEL
bg = BaggingRegressor(base_estimator=DecisionTreeRegressor(random_state=1), random_state=1, n_estimators=200, bootstrap=True, bootstrap_features=False, max_features=.75, max_samples=.5)
bg.fit(X_train, y_train)
bg_train_preds = cross_val_predict(bg, X_train, y_train, cv = cv)
bg_train_preds_df = pd.DataFrame(bg_train_preds)
bg_train_preds_df.to_csv("bg_train_preds.csv")

In [55]:
bg.fit(X_train, y_train)
bg_test_preds = bg.predict(X_test)
bg_test_preds_df = pd.DataFrame(bg_test_preds)
bg_test_preds_df.to_csv("bg_test_preds.csv")

In [80]:
# RANDOM FOREST MODEL
rf = RandomForestRegressor(random_state=1, criterion="absolute_error", n_estimators=100, max_leaf_nodes=2400, max_features=4, max_depth=36, oob_score=True, bootstrap=True, ccp_alpha=.075)
rf.fit(X_train, y_train)
rf_train_preds = cross_val_predict(rf, X_train, y_train, cv = cv)
rf_train_preds_df = pd.DataFrame(rf_train_preds)
rf_train_preds_df.to_csv("rf_train_preds.csv")

In [81]:
rf.fit(X_train, y_train)
rf_test_preds = rf.predict(X_test)
rf_test_preds_df = pd.DataFrame(rf_test_preds)
rf_test_preds_df.to_csv("rf_test_preds.csv")

In [4]:
# GRADIENT BOOSTING MODEL
gb = GradientBoostingRegressor(random_state=1, n_estimators=189, max_depth=3, learning_rate=0.095, subsample=1.0, min_samples_split=7, min_samples_leaf=1, min_weight_fraction_leaf=0, loss='huber')
gb_train_preds = cross_val_predict(gb, X_train, y_train, cv = cv)
gb_train_preds_df = pd.DataFrame(rf_train_preds)
gb_train_preds_df.to_csv("gb_train_preds.csv")

In [28]:
gb.fit(X_train, y_train)
gb_test_preds = gb.predict(X_test)
gb_test_preds_df = pd.DataFrame(gb_test_preds)
gb_test_preds_df.to_csv("gb_test_preds.csv")

In [8]:
# XGB MODEL
xg = xgb.XGBRegressor(max_depth=7,n_estimators=1500,learning_rate=0.01, reg_lambda= 100,
                         random_state=1,subsample=0.625,gamma=10)
xg_train_preds = cross_val_predict(xg, X_train, y_train, cv = cv)
xg_train_preds_df = pd.DataFrame(xg_train_preds)
xg_train_preds_df.to_csv("xg_train_preds.csv")

[04:17:00] WARNING: ../src/learner.cc:767: 
Parameters: { "loss" } are not used.

[04:18:00] WARNING: ../src/learner.cc:767: 
Parameters: { "loss" } are not used.

[04:18:34] WARNING: ../src/learner.cc:767: 
Parameters: { "loss" } are not used.

[04:19:28] WARNING: ../src/learner.cc:767: 
Parameters: { "loss" } are not used.

[04:20:02] WARNING: ../src/learner.cc:767: 
Parameters: { "loss" } are not used.



In [13]:
xg.fit(X_train, y_train)
xg_test_preds = xg.predict(X_test)
xg_test_preds_df = pd.DataFrame(xg_test_preds)
xg_test_preds_df.to_csv("xg_test_preds.csv")

[04:22:45] WARNING: ../src/learner.cc:767: 
Parameters: { "loss" } are not used.



In [82]:
# read in test predictions
ma_test_preds_csv = pd.read_csv("ma_test_preds.csv")
bg_test_preds_csv = pd.read_csv("bg_test_preds.csv")
rf_test_preds_csv = pd.read_csv("rf_test_preds.csv")
gb_test_preds_csv = pd.read_csv("gb_test_preds.csv")
xg_test_preds_csv = pd.read_csv("xg_test_preds.csv")
ma_test_preds = np.asarray(ma_test_preds_csv["0"])
bg_test_preds = np.asarray(bg_test_preds_csv["0"])
rf_test_preds = np.asarray(rf_test_preds_csv["0"])
gb_test_preds = np.asarray(gb_test_preds_csv["0"])
xg_test_preds = np.asarray(xg_test_preds_csv["0"])

In [83]:
# base model MAEs for future comparison
print("MARS MAE:")
print(mean_absolute_error(y_test, ma_test_preds))
print("Bagging MAE:")
print(mean_absolute_error(y_test, bg_test_preds))
print("RandomForest MAE:")
print(mean_absolute_error(y_test, rf_test_preds))
print("GradientBooster MAE:")
print(mean_absolute_error(y_test, gb_test_preds))
print("XGB MAE:")
print(mean_absolute_error(y_test, xg_test_preds))

MARS MAE:
36966.12909507691
Bagging MAE:
35962.31408492376
RandomForest MAE:
36044.76051390568
GradientBooster MAE:
36033.63512253498
XGB MAE:
35679.131675937126


In [84]:
# compile train and test predictors for metamodels
all_test_preds = np.transpose(np.stack([ma_test_preds, bg_test_preds, rf_test_preds, gb_test_preds, xg_test_preds]))
all_train_preds = np.transpose(np.stack([ma_train_preds, bg_train_preds, rf_train_preds, gb_train_preds, xg_train_preds]))

In [131]:
# VOTING MODEL
meta_vt_test_preds = np.mean(all_test_preds, axis=1)
meta_vt_train_preds = np.mean(all_train_preds, axis=1)
print(mean_absolute_error(y_test, meta_vt_test_preds))

35671.828809594466


In [123]:
# STACKED LINEAR MODEL
meta_ln = LinearRegression()
meta_ln.fit(all_train_preds, y_train)
meta_ln_test_preds = meta_ln.predict(all_test_preds)
print(mean_absolute_error(y_test, meta_ln_test_preds))

35758.66450925252


In [133]:
meta_ln_train_preds = cross_val_predict(meta_ln, all_train_preds, y_train, cv = cv)

In [125]:
# STACKED RANDOM FOREST MODEL
meta_rf_params = {"max_features": [1,2,3], "max_depth": [2, 5, 10], "bootstrap": [True, False], "ccp_alpha": [.001, .01, .1]}
meta_rf = RandomForestRegressor(random_state=1)
meta_rf_grid = GridSearchCV(meta_rf, meta_rf_params, scoring="neg_mean_absolute_error", n_jobs=-1, cv=cv)
meta_rf_grid.fit(all_train_preds, y_train)
print(meta_rf_grid.best_params_)
meta_rf_test_preds = meta_rf_grid.predict(all_test_preds)
print(mean_absolute_error(y_test, meta_rf_test_preds))

{'bootstrap': True, 'ccp_alpha': 0.1, 'max_depth': 10, 'max_features': 1}
35606.6262933011


In [126]:
meta_rf = RandomForestRegressor(random_state=1, bootstrap=True, ccp_alpha=.1, max_depth=10, max_features=1)
meta_rf_train_preds = cross_val_predict(meta_rf, all_train_preds, y_train, cv = cv)

In [127]:
# STACKED XGBOOST MODEL
meta_xg_params = {"subsample": [.25, .5, .75, 1], "learning_rate": [.001, .01, .1], "alpha": [.05, .1, .5, 1]}
meta_xg = xgb.XGBRegressor(random_state=1)
meta_xg_grid = GridSearchCV(meta_xg, meta_xg_params, scoring="neg_mean_absolute_error", n_jobs=-1, cv=cv)
meta_xg_grid.fit(all_train_preds, y_train)
print(meta_xg_grid.best_params_)
meta_xg_test_preds = meta_xg_grid.predict(all_test_preds)
print(mean_absolute_error(y_test, meta_xg_test_preds))

{'alpha': 0.5, 'learning_rate': 0.1, 'subsample': 1}
36157.0764871618


In [128]:
meta_xg = xgb.XGBRegressor(random_state=1, alpha=.1, learning_rate=.1, subsample=1)
meta_xg_train_preds = cross_val_predict(meta_xg, all_train_preds, y_train, cv = cv)

In [129]:
# VOTING METAMETA MODEL
meta_test_preds = np.transpose(np.stack([meta_vt_test_preds, meta_ln_test_preds.flatten(), meta_rf_test_preds]))
metameta_test_preds = np.mean(meta_test_preds, axis=1)
print(mean_absolute_error(metameta_test_preds, y_test))

35569.19911845313


In [134]:
# METAMETA LINEAR REGRESSION MODEL
meta_extra_test_preds = np.transpose(np.stack([meta_vt_test_preds, meta_ln_test_preds.flatten(), meta_rf_test_preds, meta_xg_test_preds]))
meta_train_preds = np.transpose(np.stack([meta_vt_train_preds, meta_ln_train_preds.flatten(), meta_rf_train_preds, meta_xg_train_preds]))
metameta_ln_model = LinearRegression().fit(meta_train_preds, y_train)
metameta_ln_test_preds = metameta_ln_model.predict(meta_extra_test_preds)
print(mean_absolute_error(metameta_ln_test_preds, y_test))

35510.69402672526


In [138]:
# METAMETA RANDOM FOREST MODEL
# metameta_rf_params = {"max_features": [1,2,3], "max_depth": [5, 10, 20], "oob_score": [True, False], "ccp_alpha": [.001, .01, .1]}
# metameta_rf = RandomForestRegressor(random_state=1, bootstrap=True)
# metameta_rf_grid = GridSearchCV(metameta_rf, metameta_rf_params, scoring="neg_mean_absolute_error", n_jobs=-1, cv=cv)
# metameta_rf_grid.fit(meta_train_preds, y_train)
# print(metameta_rf_grid.best_params_)
# metameta_rf_test_preds = metameta_rf_grid.predict(meta_extra_test_preds)
# print(mean_absolute_error(y_test, metameta_rf_test_preds))

In [136]:
col1 = 'model_type'
list1 = ['MARS', 'BaggingRegressor', 'RandomForestRegressor', 'GradientBoostingRegressor', 'XGBRegressor', 'Voting', 'LinearRegression', 'RandomForestRegressor', 'XGBRegressor', 'Voting', 'LinearRegression']
col2 = "model"
list2 = [ma, bg, rf, gb, xg, np.mean, meta_ln, meta_rf, meta_xg, np.mean, metameta_ln_model]
col3 = 'test_predictions'
list3 = [ma_test_preds, bg_test_preds, rf_test_preds, gb_test_preds, xg_test_preds, meta_vt_test_preds, meta_ln_test_preds, meta_rf_test_preds, meta_xg_test_preds, metameta_test_preds, metameta_ln_test_preds]
col4 = 'model_level'
list4 = [1,1,1,1,1,2,2,2,2,3,3]
all_models = pd.DataFrame(data = {col1:list1, col2:list2, col4:list4})
all_models["MAE"] = list(map(lambda x: mean_absolute_error(x, y_test), list3))
all_models["RMSE"] = list(map(lambda x: np.sqrt(mean_squared_error(x, y_test)), list3))
all_models

,model_type,model,model_level,MAE,RMSE
0,MARS,"Earth(max_degree=1, max_terms=50)",1,36966.129095,49756.778999
1,BaggingRegressor,(DecisionTreeRegressor(random_state=1028862084...,1,35962.314085,49008.562799
2,RandomForestRegressor,"(DecisionTreeRegressor(ccp_alpha=0.075, criter...",1,36044.760514,50005.958054
3,GradientBoostingRegressor,([DecisionTreeRegressor(criterion='friedman_ms...,1,36033.635123,49690.104180
4,XGBRegressor,"XGBRegressor(base_score=None, booster=None, ca...",1,35679.131676,49125.461608
5,Voting,<function mean at 0x7ff4ea7c12d0>,2,35671.828810,48979.558239
6,LinearRegression,LinearRegression(),2,35758.664509,48984.964197
7,RandomForestRegressor,"RandomForestRegressor(ccp_alpha=0.1, max_depth...",2,35606.626293,48882.229066
8,XGBRegressor,"XGBRegressor(alpha=0.1, base_score=None, boost...",2,36157.076487,49247.803209
9,Voting,<function mean at 0x7ff4ea7c12d0>,3,35569.199118,48816.591080


In [143]:
y_test_array = np.asarray(y_test).flatten()
final = pd.DataFrame(data = {"predicted": metameta_ln_test_preds.flatten(), "y_true": y_test_array})
final.to_csv("final_predictions.csv")

In [2]:
import nbformat

# Load the first notebook
with open('Project_Code_4.ipynb', 'r', encoding="utf8") as file:
    nb1 = nbformat.read(file, as_version=nbformat.NO_CONVERT)

# Load the second notebook
with open('ensembling_models (2).ipynb', 'r', encoding="utf8") as file:
    nb2 = nbformat.read(file, as_version=nbformat.NO_CONVERT)

# Create a new notebook
merged_nb = nbformat.v4.new_notebook()

# Combine the cells from both notebooks into the new notebook
merged_nb.cells.extend(nb1.cells)
merged_nb.cells.extend(nb2.cells)

# Save the merged notebook
with open('Project_Code_4.ipynb', 'w', encoding="utf8") as file:
    nbformat.write(merged_nb, file)